# Evaluation of tsdr on localization

In [1]:
%reload_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', FutureWarning)

import sys
sys.path.append("../")

import logging
logger = logging.getLogger()
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.WARN)
    logger.addHandler(handler)
    logger.setLevel(logging.WARN)

In [3]:
SOCKSHOP_DATASET_ID = "9n6mf"
TRAINTICKET_DATASET_ID = "m9dgg"

from meltria.priorknowledge.priorknowledge import SockShopKnowledge

pk = SockShopKnowledge(
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewares": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)

In [13]:
from notebooklib import localization

results_all = localization.grid_dataset_with_multi_diag_options([
    ("hierarchy-single_sbd_medoid_only_ctnrs", localization.DiagTargetPhaseOption.FIRST),  # skip phase2
    ("hierarchy-single_sbd_medoid_only_ctnrs", localization.DiagTargetPhaseOption.RAW),  # no reduction
    "hierarchy-single_sbd_medoid_only_ctnrs",
    "hdbscan_sbd_only_ctnrs",
    "fluxrank_pearsonr_medoid_only_ctnrs",
    # "fluxrank_pearsonr_max_cluster_only_ctnrs",
    "fluxrank_sbd_medoid_only_ctnrs",
    # "hdbscan_sbd_max_cluster_only_ctnrs",
    "phase2only_hierarchy-single_sbd_medoid_only_ctnrs",
    "phase1only_kstest_only_ctnrs",
    "phase1only_ediagnosis_only_ctnrs",
    "phase1only_fluxinfer_only_ctnrs",
    "kstest_hierarchy-single_sbd_medoid_only_services",
    "ediagnosis_hierarchy-single_sbd_medoid_only_services",
    "fluxinfer_hierarchy-single_sbd_medoid_only_services",
], SOCKSHOP_DATASET_ID, n=10, list_of_diag_options=[dict(
    enable_prior_knowledge=True,
    use_call_graph=True,
    use_complete_graph=False,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=False,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=True,
    use_call_graph=False,
    use_complete_graph=False,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=False,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=False,
    use_call_graph=False,
    use_complete_graph=True,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=False,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=False,
    use_call_graph=False,
    use_complete_graph=False,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=False,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=False,
    use_call_graph=False,
    use_complete_graph=False,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=True,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=True,
    use_call_graph=False,
    use_complete_graph=False,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=True,
    walk_method="monitorrank",
    root_metric_type="latency",
),
])

Loading 9n6mf with hierarchy-single_sbd_medoid_only_ctnrs...
Processing 9n6mf with hierarchy-single_sbd_medoid_only_ctnrs...
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-173
no cause metrics: 9n6mf, sock-shop, pod-network-latency/payment/2
no cause metrics: 9n6mf, sock-shop, pod-network-latency/payment/2
no cause metrics: 9n6mf, sock-shop, pod-network-latency/payment/2
no cause metrics: 9n6mf, sock-shop, pod-network-latency/payment/2
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 67 operations to synchronize with Neptune. Do not kill this process.
All 67 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-173/metadata
Loading 9n6mf with hierarchy-single_sbd_medoid_only_ctnrs...
Processing 9n6mf with hierarchy-single_sbd_medoid_only_ctnrs...
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-174


## TrainTicket Services

In [ ]:
from notebooklib import localization

results_all_trainticket = localization.grid_dataset_with_multi_diag_options([
    ("hierarchy-single_sbd_medoid_only_ctnrs", localization.DiagTargetPhaseOption.FIRST),  # skip phase2
    ("hierarchy-single_sbd_medoid_only_ctnrs", localization.DiagTargetPhaseOption.RAW),  # no reduction
    "hierarchy-single_sbd_medoid_only_ctnrs",
    "hdbscan_sbd_only_ctnrs",
    "fluxrank_pearsonr_medoid_only_ctnrs",
    # "fluxrank_pearsonr_max_cluster_only_ctnrs",
    "fluxrank_sbd_medoid_only_ctnrs",
    # "hdbscan_sbd_max_cluster_only_ctnrs",
    "phase2only_hierarchy-single_sbd_medoid_only_ctnrs",
    "phase1only_kstest_only_ctnrs",
    "phase1only_ediagnosis_only_ctnrs",
    "phase1only_fluxinfer_only_ctnrs",
    "kstest_hierarchy-single_sbd_medoid_only_services",
    "ediagnosis_hierarchy-single_sbd_medoid_only_services",
    "fluxinfer_hierarchy-single_sbd_medoid_only_services",
], TRAINTICKET_DATASET_ID, n=10, list_of_diag_options=[dict(
    enable_prior_knowledge=True,
    use_call_graph=True,
    use_complete_graph=False,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=False,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=True,
    use_call_graph=False,
    use_complete_graph=False,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=False,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=False,
    use_call_graph=False,
    use_complete_graph=True,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=False,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=False,
    use_call_graph=False,
    use_complete_graph=False,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=False,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=False,
    use_call_graph=False,
    use_complete_graph=False,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=True,
    walk_method="monitorrank",
    root_metric_type="latency",
), dict(
    enable_prior_knowledge=True,
    use_call_graph=False,
    use_complete_graph=False,
    pc_library="pcalg",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="fisher-z",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    use_indep_test_instead_of_ci=True,
    walk_method="monitorrank",
    root_metric_type="latency",
),
])